
# Local


In [1]:
! pip install --quiet -r ../requirements.txt

import sys
import os
sys.path.append(os.path.expanduser("~/repos/NeurOps/pytorch"))


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


# Colab


In [2]:
# ! git clone https://github.com/SuReLI/NeurOps.git
# ! git clone https://github.com/k8lion/GrowAndPrune.git
# %cd GrowAndPrune
# ! git pull
# %cd src
# ! pip install --quiet -r ../requirements.txt
# ! mkdir ../../data
# ! wget -O ../../data/Galaxy10_DECals.h5 wget https://astro.utoronto.ca/~hleung/shared/Galaxy10/Galaxy10_DECals.h5

# import sys
# import os
# sys.path.append("../../NeurOps/pytorch")

# Imports

In [3]:
import numpy as np
import torch
import copy
import math
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML
from collections import OrderedDict

from neurops import *

from growprune import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/kaitlin/repos/GrowAndPrune/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kaitlin/repos/GrowAndPrune/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
# ! wget -O ../../data/vgg11-8a719046.pth https://download.pytorch.org/models/vgg11-8a719046.pth 

In [5]:
copy_linear = True
limit = 15 if not copy_linear else 19
model = ModVGG11(num_classes=10, avgpooldim=7).to(device)
weights = torch.load("../../data/vgg11-8a719046.pth")
renamed = OrderedDict()
for i, (key, value) in enumerate(zip(model.state_dict(), weights.values())):
    if i <= limit:
        renamed[key] = value
    else:
        renamed[key] = model.state_dict()[key]
model.load_state_dict(renamed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
train_loader, validation_loader, test_loader = get_galaxy10_dataloaders(path_to_dir="../..", batch_size=-1, dim=224)

In [7]:
# test = torch.randn(1, 3, 64, 64).to(device)
# for layer in model:
#     test = layer(test)
#     print(test.shape)

In [8]:
test(model, validation_loader, criterion, device=device)
init_score = np.zeros(len(model.activations))
for i in range(len(model)-1):
    tensor = model.activations[str(i)]
    print(torch.transpose(torch.transpose(tensor, 0, 1).reshape(tensor.shape[1], -1), 0, 1).shape)
    init_score[i] = effective_rank(model.activations[str(i)], limit_ratio = 10)
    print(f"Layer {i} has effective rank {init_score[i]}")

train(model, train_loader, optimizer, criterion, epochs=1, val_loader=validation_loader, device=device, verbose=True)

Average loss: 0.0075, Accuracy: 160/15963 (1.00%)
torch.Size([1605632, 64])
Layer 0 has effective rank 64.0
torch.Size([802816, 128])
Layer 1 has effective rank 126.0
torch.Size([1605632, 256])
Layer 2 has effective rank 253.0
torch.Size([401408, 256])
Layer 3 has effective rank 256.0
torch.Size([802816, 512])
Layer 4 has effective rank 511.0
torch.Size([200704, 512])
Layer 5 has effective rank 504.0
torch.Size([200704, 512])
Layer 6 has effective rank 507.0
torch.Size([50176, 512])
Layer 7 has effective rank 484.0
torch.Size([1596, 4096])
Layer 8 has effective rank 1596.0
torch.Size([1596, 4096])
Layer 9 has effective rank 1537.0
Train Epoch: 0 [0/15963 (0%)]	Loss: 2.368692
Train Epoch: 0 [3200/15963 (22%)]	Loss: 1.115528


In [ ]:
#modded_model_grow = copy.deepcopy(model)
#modded_optimizer_grow = torch.optim.SGD(modded_model_grow.parameters(), lr=0.01)
#modded_optimizer_grow.load_state_dict(optimizer.state_dict())

modded_model_grow = model
modded_optimizer_grow = optimizer

for iter in range(5):
    for i in range(len(modded_model_grow)-1):
        max_rank = modded_model_grow[i].out_features if i > modded_model_grow.conversion_layer else modded_model_grow[i].out_channels
        score = effective_rank(modded_model_grow.activations[str(i)], limit_ratio = 100)
        num_to_prune = max(int(0.9*init_score[i])-score, 0)
        scores = svd_score(modded_model_grow.activations[str(i)], limit_ratio = 100)
        to_prune = np.argsort(scores.cpu().detach().numpy())[:num_to_prune]
        to_add = max(score-int(0.97*init_score[i]), 0)
        print("Layer {} score: {}/{}, neurons to prune: {}, # neurons to add: {}".format(i, score, max_rank, to_prune, to_add))
        modded_model_grow.prune(i, to_prune, optimizer=modded_optimizer_grow)
        modded_model_grow.grow(i, to_add, fanin_weights="iterative_orthogonalization", 
                               optimizer=modded_optimizer_grow)
    print("The modded model now has {} effective parameters.".format(modded_model_grow.parameter_count(masked = True)))
    print("Validation after growing: ", end = "")
    test(modded_model_grow, validation_loader, criterion, device=device)
    train(modded_model_grow, train_loader, modded_optimizer_grow, criterion, epochs=2, val_loader=validation_loader, device=device)

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 2498119335936 bytes. Error code 12 (Cannot allocate memory)